## 超纲词检测

In [ ]:
import functools
import re
import ipywidgets as widgets
from src.dict_process import *
from src.check_word import get_words_out_of_dict, all_variant_in_dict, check_in_dict

In [ ]:
def fix_dict(Dictionary):
    Dictionary["others"] = Dictionary["other"]
    Dictionary["toward"] = Dictionary["towards"]
    Dictionary["safe"] = [v for v in Dictionary["safe"] if not v[3].startswith("初")]
    for w in ["", "re", "non", "dis"]:
        if w not in Dictionary:
            Dictionary[w] = [("","","","")]

In [ ]:
Dictionary = load_dict("src/dictionary.jsonl")
fix_dict(Dictionary)
RevDicts = get_rev_dicts(Dictionary)
Series = sorted(get_series_list(RevDicts))
if Series[0] == "":
    Series = Series[1:] + Series[:1]
global Sorted_Names, L1, L2s
Disable_update = False

def update_global_level_params(kept_series):
    if not Disable_update:
        global Sorted_Names, L1, L2s
        Sorted_Names = tuple(get_sorted_kept_names(RevDicts, kept_series))
        L1, L2s = get_levels(Sorted_Names)

@functools.lru_cache
def get_filtered_vocabulary_RevDicts(Sorted_Names, level_begin, level_end):
    return get_filtered_vocabulary(RevDicts, Sorted_Names, level_begin, level_end)
def get_vocabulary():
    level_begin = (dd_low1.value,dd_low2.value)
    level_end = (dd_upp1.value,dd_upp2.value)
    vocabulary = get_filtered_vocabulary_RevDicts(Sorted_Names, level_begin, level_end)
    return vocabulary

In [ ]:
def series_toggle_all(change):
    global Disable_update
    Disable_update= True
    for cb in series_checkboxes:
        cb.value = change["new"]
    Disable_update = False
    update_dd1_options("", dd_low1, series_checkboxes, 0)
    update_dd1_options("", dd_upp1, series_checkboxes, -1)
    
    
cb_series_all = widgets.Checkbox(value=True, description="All"+"\xa0"*4, disabled=False, indent=False, layout=dict(width='auto'))
cb_series_all.observe(series_toggle_all, 'value')

In [ ]:
series_checkboxes = [
    widgets.Checkbox(
        value=True,
        description=(s if s else "UNKNOWN")+"\xa0"*4,
        disabled=False,
        indent=False,
        layout=dict(width='auto'),
    ) for s in Series]


In [ ]:
def update_dd1_options(change, dropdown, series_checkboxes, value_ind=0):
    kept_series = [se for se, ch in zip(Series, series_checkboxes) if ch.value]
    update_global_level_params(kept_series)
    dropdown.options = L1
    dropdown.value = dropdown.options[value_ind] if dropdown.options else None
    update_vocab_len()
for wi in series_checkboxes:
    wi.observe(lambda ch: update_dd1_options(ch, dd_low1, series_checkboxes, 0), 'value')
    wi.observe(lambda ch: update_dd1_options(ch, dd_upp1, series_checkboxes, -1), 'value')
series_widget = widgets.Box(children=[widgets.Label("教材："), cb_series_all] + series_checkboxes)
series_widget

In [ ]:
dd_low1 = widgets.Dropdown(
    description='最低:',
    disabled=False,
    layout=dict(width='auto'),
    style={'description_width': 'initial'}
)
dd_low2 = widgets.Dropdown(
    description='',
    disabled=False,
    layout=dict(width='auto'),
)
dd_upp1 = widgets.Dropdown(
    description='最高:',
    disabled=False,
    layout=dict(width='auto'),
)
dd_upp2 = widgets.Dropdown(
    description='',
    disabled=False,
    layout=dict(width='auto'),
)
lb_vocablen = widgets.Label("", layout={"padding":'0px 0px 0px 30px'})

def update_dd2_options(change, dropdown, value_ind=0):
    dropdown.options = L2s[change["new"]] if change["new"] is not None else []
    dropdown.value = dropdown.options[value_ind] if dropdown.options else None

def update_vocab_len(*a):
    l = len(get_vocabulary())
    lb_vocablen.value = "词汇量: %d"%l
    
dd_low1.observe(lambda ch: update_dd2_options(ch, dd_low2, 0), 'value')
dd_upp1.observe(lambda ch: update_dd2_options(ch, dd_upp2, -1), 'value')
dd_low1.observe(update_vocab_len)
dd_low2.observe(update_vocab_len)
dd_upp1.observe(update_vocab_len)
dd_upp2.observe(update_vocab_len)
update_dd1_options(None, dd_low1, series_checkboxes, 0)
update_dd1_options(None, dd_upp1, series_checkboxes, -1)


In [ ]:
widgets.Box([widgets.Box(children=[dd_low1, dd_low2]), widgets.Box(children=[dd_upp1, dd_upp2]), lb_vocablen])

In [ ]:
def lemmatize_toggle_all(change):
    for cb in lemmatize_checkboxes:
        cb.value = change["new"]
    
cb_lemmatize_all = widgets.Checkbox(value=True, description="All"+"\xa0"*4, disabled=False, indent=False, layout=dict(width='auto'))
cb_lemmatize_all.observe(lemmatize_toggle_all, 'value')

lem_tags = {"n":"nouns", "v":"verbs", "a":"adjectives", "r":"adverbs", "s":"satellite adjectives"}
lemmatize_checkboxes = [
    widgets.Checkbox(
        value=True,
        description=t+"\xa0"*4,
        disabled=False,
        indent=False,
        layout=dict(width='auto'),
    ) for t in lem_tags.values()]
lemmatize_widget = widgets.HBox(children=[widgets.Label("屈折变化："), cb_lemmatize_all] + lemmatize_checkboxes)
lemmatize_widget

In [ ]:
import codecs
file = widgets.FileUpload(multiple=False, description="Load File", accept=".txt", layout=dict(width='20%', min_width="max-content"))
def on_upload_change(*a):
    c = get_content(file)
    if c:
        text_widget.value = c
        on_button_clicked(button)
file.observe(on_upload_change, 'value')

def get_content(file):
    file = file.value
    if isinstance(file, dict):
        c = next(iter(file.values()))['content']
    else:
        c = file[0]['content']
    return codecs.decode(c, encoding="utf-8")

In [ ]:
import sys
class MyWriter:
    def __init__(self, file):
        self.logfile = file
    def write(self, text):
        sys.stdout.write(text)
        self.logfile.write(text)
    def close(self):
        sys.stdout.close()
        self.logfile.close()
        
button = widgets.Button(description="检测", button_style="success", 
                        icon="arrow-alt-circle-right", layout=dict(width='30%', min_width="max-content"))
out = widgets.Output()

@out.capture()
def on_button_clicked(b):
    out.clear_output()
    with open("output-words_excluded.txt", 'w', encoding="utf-8") as f:
        io = MyWriter(f)
        out_words = get_out_words()
        show_out_words(out_words, io=io)
        out_words_in_dict = get_out_words_in_dict(out_words)
        owords = [w for w in out_words if w not in set(out_words_in_dict)]
        show_text(text_widget.value, [owords, out_words_in_dict])
        if out_words_in_dict:
            print("\n其中已登录的超纲词%d个："%(len(out_words_in_dict)), file=io)
            print(", ".join(out_words_in_dict), file=io)
            words_widget.value = ", ".join(flatten_words(out_words_in_dict))
            on_button_clicked2(button2)
    
button.on_click(on_button_clicked)

In [ ]:
import html
def color_style(color):
    return lambda w: f"<span style='color:{color}'>{w}</span>"
style_list=[color_style(c) for c in ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]]
def multi_highlight(text, words_list, style_list=style_list):
    def highlight(w):
        w = w.group(0)
        for i,words in enumerate(words_list):
            if w in words:
                return style_list[i%len(style_list)](w)
        return w
    return re.sub(r"\b([\w\.\-]+)\b", highlight, text)

text_out = widgets.Output()
@text_out.capture()          
def show_text(text, *args):
    text_out.clear_output()
    hd = '<p style="white-space:pre-wrap;text-align:left;">'
    ta = '</p>'
    display(HTML(hd+ multi_highlight(html.escape(text), *args) + ta))

In [ ]:
def get_out_words():
    tags = [t for t,c in zip(lem_tags, lemmatize_checkboxes) if c.value]
    out_words = get_words_out_of_dict(text_widget.value, get_vocabulary(), tags=tags)
    return out_words

def show_out_words(out_words, io):
    if out_words:
        print("超纲词%d个："%(len(out_words)), file=io)
        print( ", ".join(out_words), file=io)
    else:
        print("没有超纲词。", file=io)
def get_out_words_in_dict(out_words):
    out_words_in_dict = []
    for w1 in out_words:
        isindict, w2 = check_in_dict(w1, Dictionary)
        if isindict:
            out_words_in_dict.append(w1)
    return out_words_in_dict
def flatten_words(ws):
    l = []
    for w in ws:
        if isinstance(w, str):
            l.append(w)
        else:
            l.extend(flatten_words(w))
    return l

In [ ]:
text_widget = widgets.Textarea(placeholder="请输入要检测的文本", layout=dict(width='auto', height='10em'))
text_widget

In [ ]:
widgets.HBox(children=[button, file])

In [ ]:
out

In [ ]:
widgets.Accordion([text_out], selected_index=0)

In [ ]:
from IPython.display import HTML, display

def display_table(data):
    html = '<table>'
    for row in data:
        html += '<tr>'
        if row:
            html += '<td><h4>%s</h4></td>'%(row[0])
        for field in row[1:]:
            html += '<td style="padding-left:4em"><h4>%s</h4></td>'%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

In [ ]:
words_widget = widgets.Text(
    value='',
    placeholder='输入要查询的单词，用逗号分隔',
    disabled=False,
    layout=dict(width='40%', min_width="max-content"),
    continuous_update=False,
)


In [ ]:
query_out = widgets.Output()
button2 = widgets.Button(description="查询",
                         button_style="success",
                         icon="magnifying-glass",
                         layout=dict(width='10%', min_width="max-content"))

@query_out.capture()          
def on_button_clicked2(b):
    query_out.clear_output()
    notfound = []
    words_all = words_widget.value.split(",")
    for w in list(dict.fromkeys(words_all)):
        w = w.strip()
        w2s = all_variant_in_dict(w, Dictionary)
        w2s or notfound.append(w)
        for w2 in w2s:
            display_table(Dictionary[w2])
        display(HTML("<hr>"))
    if notfound:
        print(",".join([repr(w) for w in notfound]), "未查询到（%d个）。"%(len(notfound)) if len(notfound)>5 else "未查询到。")
    
button2.on_click(on_button_clicked2)
words_widget.observe(on_button_clicked2, "value")

In [ ]:
search_widget = widgets.HBox(children=[words_widget, button2])
search_widget

In [ ]:
widgets.Accordion([query_out], selected_index=0)